# DeepMIR

## 1. Load tool packages

In [3]:
from Metrics import Metrics
import wget
from Predict import predict

d:\anaconda3\envs\IRFormer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load the data

In [4]:
wget.download('https://github.com/LinTan-CSU/DeepMIR/releases/download/OrganicSolventsDataset/Quaternary.npy', 'Quaternary.npy')
wget.download('https://github.com/LinTan-CSU/DeepMIR/releases/download/OrganicSolventsDataset/Quaternary_labels.csv', 'Quaternary_labels.csv')

'Quaternary_labels.csv'

## 3. Component Prediction

In [5]:
test_labels = ['1,2-dichloroethane', '1-butanol', 'acetonitrile', 'cyclohexane', 'dichloromethane', 'diethylene_glycol_dimethyl_ether', 'ethanol', 'hexane', 'isopropyl_alcohol', 'methanol', 'toluene', 'trichloromethane']
predict(model_path='DeepMIR.pth', test_dataset_path='Quaternary.npy', test_labels=test_labels, output_path='DeepMIR.csv')

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,diethylene_glycol_dimethyl_ether,0.999988,"1,2-dichloroethane",0.999822,cyclohexane,0.998484,trichloromethane,0.588363,hexane,0.000586,...,toluene,0.000111,ethanol,6.525464e-07,isopropyl_alcohol,3.404371e-10,acetonitrile,2.761313e-13,1-butanol,3.925265e-18
1,diethylene_glycol_dimethyl_ether,0.999663,cyclohexane,0.956367,dichloromethane,0.837904,hexane,0.527475,1-butanol,0.082233,...,ethanol,0.000878,toluene,2.358339e-04,isopropyl_alcohol,1.074001e-04,"1,2-dichloroethane",2.463072e-06,acetonitrile,1.320689e-09
2,isopropyl_alcohol,0.999952,cyclohexane,0.971515,acetonitrile,0.969892,toluene,0.218512,dichloromethane,0.010513,...,ethanol,0.001735,trichloromethane,1.202279e-03,methanol,1.005918e-03,1-butanol,4.266664e-04,"1,2-dichloroethane",8.294266e-06
3,"1,2-dichloroethane",0.998941,diethylene_glycol_dimethyl_ether,0.987858,acetonitrile,0.939851,cyclohexane,0.839800,methanol,0.486477,...,trichloromethane,0.000421,dichloromethane,3.680389e-04,isopropyl_alcohol,2.897393e-04,toluene,1.416228e-04,1-butanol,3.206478e-10
4,diethylene_glycol_dimethyl_ether,0.999968,"1,2-dichloroethane",0.999639,cyclohexane,0.995816,trichloromethane,0.602842,methanol,0.001158,...,toluene,0.000115,ethanol,1.650532e-06,isopropyl_alcohol,8.487563e-10,acetonitrile,8.625079e-12,1-butanol,4.567917e-17
5,"1,2-dichloroethane",0.998719,ethanol,0.998365,cyclohexane,0.978304,dichloromethane,0.199641,trichloromethane,0.002726,...,diethylene_glycol_dimethyl_ether,0.001170,acetonitrile,3.741393e-04,toluene,2.245183e-04,isopropyl_alcohol,7.064365e-06,1-butanol,1.125951e-06
6,diethylene_glycol_dimethyl_ether,0.999997,"1,2-dichloroethane",0.999967,hexane,0.923340,trichloromethane,0.529294,methanol,0.000365,...,cyclohexane,0.000032,ethanol,2.784371e-06,isopropyl_alcohol,2.330597e-08,acetonitrile,6.985451e-13,1-butanol,1.424928e-16
7,isopropyl_alcohol,0.999983,cyclohexane,0.983975,acetonitrile,0.865346,dichloromethane,0.360558,hexane,0.007994,...,ethanol,0.002138,methanol,1.061631e-03,toluene,1.693568e-04,1-butanol,4.885851e-06,"1,2-dichloroethane",3.932366e-06
8,diethylene_glycol_dimethyl_ether,0.999989,cyclohexane,0.994922,methanol,0.844187,trichloromethane,0.501955,hexane,0.000747,...,toluene,0.000058,isopropyl_alcohol,1.274932e-06,1-butanol,1.355175e-07,"1,2-dichloroethane",8.200024e-08,acetonitrile,1.334244e-10
9,"1,2-dichloroethane",0.988311,cyclohexane,0.968752,diethylene_glycol_dimethyl_ether,0.968052,toluene,0.304382,dichloromethane,0.002067,...,methanol,0.000600,ethanol,1.023017e-06,isopropyl_alcohol,2.256274e-08,acetonitrile,3.159013e-10,1-butanol,5.406857e-15


## 4. Metrics

In [6]:
TPR, TNR, ACC = Metrics('DeepMIR.csv', 'Quaternary_labels.csv')
print("TPR:{:.4f}".format(TPR), "TNR:{:.4f}".format(TNR), "ACC:{:.4f}".format(ACC))

TPR:0.8750 TNR:0.9875 ACC:0.9500
